Tutorial : https://medium.com/@nishantsahoo/which-movie-should-i-watch-5c83a3c0f5b1

In [3]:
#Basic beautiful soup

import urllib3
from bs4 import BeautifulSoup
url = "https://www.rollingstone.com/tv-movies/tv-movie-lists/best-tv-shows-of-all-time-1234598313/oz-2-1234598409/"
ourUrl = urllib3.PoolManager().request('GET', url).data

soup = BeautifulSoup(ourUrl, "lxml")
print(soup.find('title').text)
print(soup.find('script').text)

Best TV Shows of All Time

			window.dataLayer = window.dataLayer || [];
			function gtag(){dataLayer.push(arguments);}

			window.pmc_google_consent_mode = {
				init: function(opts = {}) {
					/** @type string[] */
					const consentGroups = window.pmc_onetrust_helpers?.getActiveConsentGroups?.() || [];

					/**
					 * C0001 = Strictly Necessary Cookies (Security, Auth, Basic Functionality)
					 * C0002 = Performance Cookies (Analytics)
					 * C0003 = Functional Cookies (Personalization)
					 * C0004 = Targeting Cookies (Advertising)
					 */
					const configuration = {
						'ad_storage': consentGroups.includes('C0004') ? 'granted' : 'denied',
						'analytics_storage': consentGroups.includes('C0002') ? 'granted' : 'denied',
						'functionality_storage': consentGroups.includes('C0003') ? 'granted' : 'denied',
						'personalization_storage': consentGroups.includes('C0003') ? 'granted' : 'denied',
						'security_storage': consentGroups.includes('C0001') ? 'granted' : 'denied

General structure of the website https://www.rollingstone.com/tv-movies/tv-movie-lists/best-tv-shows-of-all-time-1234598313/what-we-do-in-the-shadows-3-1234598401/

In [ ]:
<div id="pmc-gallery-vertical">
    <div class="c-gallery-vertical-loader u-gallery-app-shell-loader">
        <ul class="pmc-fallback-list-items lrv-a-unstyle-list lrv-u-margin-t-2">
            <li class="pmc-fallback-list-item-wrap lrv-u-margin-b-2">
			<article class="pmc-fallback-list-item">
	            <h2>&#8216;What We Do in the Shadows&#8217;</h2>
                <div class="pmc-not-a-paywall">
                    <p class="paragraph"></p>
                </div>
            </article>
            </li>

            <li>
            </li>

In [8]:
#Scraper
from tqdm import tqdm # to display progess bar
i=100
showList=soup.findAll('li', attrs={'class':"pmc-fallback-list-item-wrap lrv-u-margin-b-2"})
for item in tqdm(showList):
    article=item.find('article', attrs={'class':'pmc-fallback-list-item'})
    title=article.find('h2')
    print(title)
    print(str(i)+'. '+str(title[0].contents[0].encode('utf-8').decode('ascii', 'ignore')))
    i-=1




  0%|          | 0/50 [00:00<?, ?it/s]

<h2>‘What We Do in the Shadows’</h2>


KeyError: 0